# Notebook 1: Data Preprocessing and Causal Graph Construction

### Goal
The purpose of this notebook is to load the raw financial data, perform necessary cleaning and feature engineering, and define the causal graph structure that will be used in our experiments. The final output will be a set of processed `.csv` files that our PyTorch Geometric `Dataset` can easily ingest.

### Steps
1.  **Load Raw Data:** Load asset returns, factor returns, and macro data from their source files.
2.  **Define the Causal Graph:**
    *   This is a critical modeling step. We will define the `edge_index` based on domain knowledge.
    *   Example structure:
        *   Edges from Macro nodes -> Factor nodes.
        *   Edges from Factor nodes -> Asset nodes.
        *   Edges between Asset nodes based on GICS sector membership.
3.  **Save Processed Data:** Save the final node feature time-series and the static edge list to the `data/processed/` directory.